In [ ]:
%matplotlib notebook
import numpy as np
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import FlatLambdaCDM
import scipy.stats
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
plt.ion()

knn = 10
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
astorad = np.pi/(180.*60.*60)
Rauto_cut = 22.9
dz_membership = 0.02

# Load the classifications and photometry

In [ ]:
classtable = Table().read('../catalogs/classified_objects.csv')
for i in range(knn):
    classtable[str(i+1)+'NN_arcsec'] = np.zeros(len(classtable))

phottable = Table().read('../catalogs/slits_phot_zs_cutonslitdist_rcl.csv')
phottable = phottable[np.where(phottable['Rauto'] < Rauto_cut)]
phottable = phottable[np.where(phottable['Q'] >= 3)]

# Find the k-NN (arcsec) and calculate the harmonic mean

In [ ]:
for row in classtable:
    # Find nearby objects in z
    subtable = phottable[np.where(abs(phottable['zLDP'] - row['zLDP']) < dz_membership)]
    subcoords = SkyCoord(ra=subtable['ra']*u.degree, dec=subtable['dec']*u.degree)

    obj = SkyCoord(ra=row['ra']*u.degree, dec=row['dec']*u.degree)
    seps = subcoords.separation(obj).arcsecond
    kseps = sorted(seps)[1:knn+1]
    for i in range(knn):
        row[str(i+1)+'NN_arcsec'] = kseps[i]

# Calculate density in galaxies/Mpc$^2$

In [ ]:
classtable['harm_mean'] = np.zeros(len(classtable))
classtable['surface_density'] = np.zeros(len(classtable))

for row in classtable:
    DA = cosmo.angular_diameter_distance(row['zLDP'])
    kseps = np.array([row[str(i+1)+'NN_arcsec'] for i in range(knn)]) * astorad * DA.value
    row['harm_mean'] = scipy.stats.hmean(kseps)
    
    # Account for the object itself!
    n_in_area = len(np.where(kseps<row['harm_mean'])[0]) + 1
    row['surface_density'] = n_in_area/(np.pi*row['harm_mean']*row['harm_mean'])

# Make the completeness correction
At the moment, this is to just divide by Q2 completeness at the relevent radius. Q2 since we are calculating the NN with Q=4,3,2 objects

In [ ]:
completeness = Table().read('../catalogs/radial_completeness.dat', format='ascii')
completeness['theta_cl_min'] = completeness['theta_cl'] - 0.25
completeness['theta_cl_max'] = completeness['theta_cl'] + 0.25

In [ ]:
classtable['surface_density_compcorred'] = np.zeros(len(classtable))
for row in classtable:
    rcl = (row['theta_cl_radian']*u.radian).to(u.arcminute).value
    for crow in completeness:
        if crow['theta_cl_min'] < rcl < crow['theta_cl_max']:
            row['surface_density_compcorred'] = row['surface_density']/crow['q3']

# Save the updated table

In [ ]:
classtable.write('../catalogs/classified_surfacedensity.csv', format='csv', overwrite=True)